In [2]:
import os
import json
import math
import numpy as np
import tensorflow as tf
import torch

import grid2op
from grid2op.Reward import *
from grid2op.Action import *

%run d3qn.ipynb
%run opponent.ipynb

In [3]:
def train_d3qn_against_no_opponent(env, agent, num_pre_training_steps, n_iter,
                                   save_path, log_path):
    # Make sure we can fill the experience buffer
    if num_pre_training_steps < agent.batch_size * agent.num_frames:
        num_pre_training_steps = agent.batch_size * agent.num_frames
        
    # Loop vars
    num_training_steps = n_iter
    num_steps = num_pre_training_steps + num_training_steps
    step = 0
    alive_steps = 0
    total_reward = 0
    agent.done = True
    print(f"Total number of steps: {num_steps}")

    # Create file system related vars
    logpath = os.path.join(log_path, agent.name)
    os.makedirs(save_path, exist_ok=True)
    modelpath = os.path.join(save_path, agent.name + ".h5")
    agent.tf_writer = tf.summary.create_file_writer(logpath, name=agent.name)
    agent._save_hyperparameters(save_path, env, num_steps)
    
    while step < num_steps:
        # Init first time or new episode
        if agent.done:
            new_obs = env.reset() # This shouldn't raise
            agent.reset(new_obs)
        if cfg.VERBOSE and step % 1000 == 0:
            print("Step [{}] -- Random [{}]".format(step, agent.epsilon))

        # Save current observation to stacking buffer
        agent._save_current_frame(agent.state)

        # Choose an action
        if step <= num_pre_training_steps:
            a = agent.Qmain.random_move()
        elif np.random.rand(1) < agent.epsilon:
            a = agent.Qmain.random_move()
        elif len(agent.frames) < agent.num_frames:
            a = 0 # Do nothing
        else:
            a, _ = agent.Qmain.predict_move(np.array(agent.frames))

        # Convert it to a valid action
        act = agent.convert_act(a)
        # Execute action
        new_obs, reward, agent.done, info = env.step(act)
        new_state = agent.convert_obs(new_obs)
        if info["is_illegal"] or info["is_ambiguous"] or \
           info["is_dispatching_illegal"] or info["is_illegal_reco"]:
            if cfg.VERBOSE:
                print (a, info)
        total_reward += reward

        # Save new observation to stacking buffer
        agent._save_next_frame(new_state)

        # Save to experience buffer
        if len(agent.frames2) == agent.num_frames:
            agent.per_buffer.add(np.array(agent.frames),
                                a, reward,
                                np.array(agent.frames2),
                                agent.done)

        # Perform training when we have enough experience in buffer
        if step >= num_pre_training_steps:
            training_step = step - num_pre_training_steps
            # Decay chance of random action
            agent.epsilon = agent._adaptive_epsilon_decay(training_step)

            # Perform training at given frequency
            if step % cfg.UPDATE_FREQ == 0 and \
               len(agent.per_buffer) >= agent.batch_size:
                # Perform training
                agent._batch_train(training_step, step)

                if cfg.UPDATE_TARGET_SOFT_TAU > 0.0:
                    tau = cfg.UPDATE_TARGET_SOFT_TAU
                    # Update target network towards primary network
                    agent.Qmain.update_target_soft(agent.Qtarget.model, tau)

            # Every UPDATE_TARGET_HARD_FREQ trainings, update target completely
            if cfg.UPDATE_TARGET_HARD_FREQ > 0 and \
               step % (cfg.UPDATE_FREQ * cfg.UPDATE_TARGET_HARD_FREQ) == 0:
                agent.Qmain.update_target_hard(agent.Qtarget.model)
        
        if agent.done:
            agent.epoch_rewards.append(total_reward)
            agent.epoch_alive.append(alive_steps)
            if cfg.VERBOSE and step > num_pre_training_steps:
                print("step {}: Survived [{}] steps".format(step, alive_steps))
                print("Total reward [{}]".format(total_reward))
            alive_steps = 0
            total_reward = 0
        else:
            alive_steps += 1
            
        ######## After Each Step #######
        if step > 0 and step % 2000 == 0: # save network every 5000 iters
            agent.save(modelpath)
        step += 1
        # Make new obs the current obs
        agent.obs = new_obs
        agent.state = new_state

    # Save model after all steps
    agent.save(modelpath)

In [5]:
# agent params
num_pre_training_steps = 256
learning_rate = 1e-4
initial_epsilon = 0.99
final_epsilon = 0.01
decay_epsilon = 20000

# training params
n_iter = 20000
env_name = "rte_case14_realistic"
env = make(env_name, reward_class=CombinedScaledReward)

# Register custom reward for training
cr = env._reward_helper.template_reward
#cr.addReward("overflow", CloseToOverflowReward(), 1.0)
cr.addReward("game", GameplayReward(), 1.0)
#cr.addReward("recolines", LinesReconnectedReward(), 1.0)
cr.addReward("l2rpn", L2RPNReward(), 2.0/float(env.n_line))
# Initialize custom rewards
cr.initialize(env)
# Set reward range to something managable
cr.set_range(-1.0, 1.0)

agent_name = "DDDQN"
save_path = "saved_agent_DDDQN_no_opponent_{}".format(n_iter)
log_path="tf_logs_DDDQN"

agent = DoubleDuelingDQN(env.observation_space, env.action_space, name=agent_name,
                         is_training=True, learning_rate=learning_rate,
                         initial_epsilon=initial_epsilon, final_epsilon=final_epsilon, decay_epsilon=decay_epsilon)

train_d3qn_against_no_opponent(env, agent, num_pre_training_steps, n_iter,
                                   save_path, log_path)

Agent action size: 141
Total number of steps: 20256
Step [0] -- Random [0.99]
step 257: Survived [2] steps
Total reward [0.4758448600769043]
step 260: Survived [2] steps
Total reward [0.5162427425384521]
step 262: Survived [1] steps
Total reward [-0.22687530517578125]
step 263: Survived [0] steps
Total reward [-1.0]
step 269: Survived [5] steps
Total reward [2.5897278785705566]
loss = 6839.189
step 280: Survived [10] steps
Total reward [7.117826581001282]
step 281: Survived [0] steps
Total reward [-1.0]
step 283: Survived [1] steps
Total reward [-0.1944117546081543]
step 285: Survived [1] steps
Total reward [-0.15723586082458496]
step 286: Survived [0] steps
Total reward [-1.0]
step 289: Survived [2] steps
Total reward [0.4938105344772339]
step 291: Survived [1] steps
Total reward [-0.2002314329147339]
step 294: Survived [2] steps
Total reward [0.4412109851837158]
step 300: Survived [5] steps
Total reward [2.792956829071045]
step 306: Survived [5] steps
Total reward [2.829564690589905]

step 638: Survived [0] steps
Total reward [-1.0]
step 641: Survived [2] steps
Total reward [0.3148399591445923]
step 644: Survived [2] steps
Total reward [0.6524664163589478]
step 646: Survived [1] steps
Total reward [-0.23400557041168213]
step 650: Survived [3] steps
Total reward [1.2080037593841553]
step 651: Survived [0] steps
Total reward [-1.0]
step 655: Survived [3] steps
Total reward [1.3051862716674805]
step 661: Survived [5] steps
Total reward [2.8659268617630005]
step 665: Survived [3] steps
Total reward [1.4436231851577759]
step 670: Survived [4] steps
Total reward [2.187295436859131]
step 671: Survived [0] steps
Total reward [-1.0]
loss = 103.28259
step 672: Survived [0] steps
Total reward [-1.0]
step 677: Survived [4] steps
Total reward [2.2079498767852783]
step 681: Survived [3] steps
Total reward [0.7469242215156555]
step 682: Survived [0] steps
Total reward [-1.0]
step 684: Survived [1] steps
Total reward [-0.2564300298690796]
step 687: Survived [2] steps
Total reward [

step 1036: Survived [5] steps
Total reward [2.8560553789138794]
step 1037: Survived [0] steps
Total reward [-1.0]
step 1041: Survived [3] steps
Total reward [1.3659497499465942]
step 1044: Survived [2] steps
Total reward [0.463711142539978]
step 1049: Survived [4] steps
Total reward [2.2953548431396484]
step 1054: Survived [4] steps
Total reward [1.9092825651168823]
step 1055: Survived [0] steps
Total reward [-1.0]
step 1060: Survived [4] steps
Total reward [1.7924312353134155]
step 1063: Survived [2] steps
Total reward [0.2592122554779053]
loss = 21.177113
step 1066: Survived [2] steps
Total reward [0.6230053901672363]
step 1068: Survived [1] steps
Total reward [-0.18503832817077637]
step 1071: Survived [2] steps
Total reward [0.46353018283843994]
step 1074: Survived [2] steps
Total reward [0.36610865592956543]
step 1083: Survived [8] steps
Total reward [5.614491820335388]
step 1084: Survived [0] steps
Total reward [-1.0]
step 1087: Survived [2] steps
Total reward [0.545758843421936]


step 1442: Survived [4] steps
Total reward [2.151926875114441]
step 1447: Survived [4] steps
Total reward [2.0646060705184937]
step 1449: Survived [1] steps
Total reward [-0.3160921335220337]
step 1452: Survived [2] steps
Total reward [0.47760283946990967]
loss = 20.266653
step 1456: Survived [3] steps
Total reward [1.2421305179595947]
step 1457: Survived [0] steps
Total reward [-1.0]
step 1460: Survived [2] steps
Total reward [0.4466984272003174]
step 1462: Survived [1] steps
Total reward [-0.18335425853729248]
step 1463: Survived [0] steps
Total reward [-1.0]
step 1464: Survived [0] steps
Total reward [-1.0]
step 1468: Survived [3] steps
Total reward [1.3160704374313354]
step 1471: Survived [2] steps
Total reward [0.5843439102172852]
step 1475: Survived [3] steps
Total reward [1.313818335533142]
step 1479: Survived [3] steps
Total reward [1.3316068649291992]
step 1481: Survived [1] steps
Total reward [-0.1960461139678955]
step 1483: Survived [1] steps
Total reward [-0.242165327072143

step 1846: Survived [1] steps
Total reward [-0.17377448081970215]
loss = 13.484483
step 1848: Survived [1] steps
Total reward [-0.2337939739227295]
step 1853: Survived [4] steps
Total reward [2.1071972846984863]
step 1855: Survived [1] steps
Total reward [-0.181196928024292]
step 1863: Survived [7] steps
Total reward [4.44002377986908]
step 1864: Survived [0] steps
Total reward [-1.0]
step 1867: Survived [2] steps
Total reward [0.43021678924560547]
step 1871: Survived [3] steps
Total reward [1.2044025659561157]
step 1875: Survived [3] steps
Total reward [1.3048458099365234]
step 1878: Survived [2] steps
Total reward [0.06549906730651855]
step 1881: Survived [2] steps
Total reward [0.5098645687103271]
step 1884: Survived [2] steps
Total reward [0.47810590267181396]
step 1889: Survived [4] steps
Total reward [2.085319757461548]
step 1892: Survived [2] steps
Total reward [0.41849982738494873]
step 1896: Survived [3] steps
Total reward [1.3851933479309082]
step 1900: Survived [3] steps
Tot

...

step 16237: Survived [19] steps
Total reward [13.591951251029968]
loss = 0.0010859963
step 16246: Survived [8] steps
Total reward [5.232967138290405]
step 16250: Survived [3] steps
Total reward [1.179023027420044]
step 16266: Survived [15] steps
Total reward [10.338696360588074]
step 16292: Survived [25] steps
Total reward [17.92546033859253]
step 16295: Survived [2] steps
Total reward [0.4298062324523926]
loss = 0.0013069786
5 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': True, 'is_ambiguous': False, 'is_dispatching_illegal': False, 'is_illegal_reco': False, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [Grid2OpException IllegalAction IllegalAction('Powerline with ids [4] have been modified illegally (cooldown)',)], 'rewards': {}}
step 16312: Survived [16] steps
Total reward [10.113874

118 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True, 'is_illegal_reco': False, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [Grid2OpException AmbiguousAction InvalidRedispatching InvalidRedispatching('You cannot ask for a dispatch higher than pmax - pmin  [it would be always invalid because, even if the sepoint is pmin, this dispatch would set it to a number higher than pmax, which is impossible]. Invalid dispatch for generator(s): [0]',)], 'rewards': {}}
118 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True, 'is_illegal_reco': 

117 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True, 'is_illegal_reco': False, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [Grid2OpException AmbiguousAction InvalidRedispatching InvalidRedispatching('You cannot ask for a dispatch higher than pmax - pmin  [it would be always invalid because, even if the sepoint is pmin, this dispatch would set it to a number higher than pmax, which is impossible]. Invalid dispatch for generator(s): [0]',)], 'rewards': {}}
loss = 0.0012755056
117 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True,

117 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True, 'is_illegal_reco': False, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [Grid2OpException AmbiguousAction InvalidRedispatching InvalidRedispatching('You cannot ask for a dispatch higher than pmax - pmin  [it would be always invalid because, even if the sepoint is pmin, this dispatch would set it to a number higher than pmax, which is impossible]. Invalid dispatch for generator(s): [0]',)], 'rewards': {}}
117 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True, 'is_illegal_reco': 

118 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True, 'is_illegal_reco': False, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [Grid2OpException AmbiguousAction InvalidRedispatching InvalidRedispatching('You cannot ask for a dispatch higher than pmax - pmin  [it would be always invalid because, even if the sepoint is pmin, this dispatch would set it to a number higher than pmax, which is impossible]. Invalid dispatch for generator(s): [0]',)], 'rewards': {}}
118 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True, 'is_illegal_reco': 